In [43]:
import os
import glob
import json
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from elasticsearch import Elasticsearch
import pandas as pd
import json
import pickle
from elasticsearch.helpers import bulk

In [5]:
df = pd.DataFrame({
        'id' : pd.Series(dtype='str'),
        'username' : pd.Series(dtype='str'),
        'created_at' : pd.Series(dtype='str'),
        'favourites_count': pd.Series(dtype='float'),
        'followers_count': pd.Series(dtype='float'),
        'friends_count' : pd.Series(dtype='float'),
        'tweet_id' : pd.Series(dtype='str'),
        'created_on' : pd.Series(dtype='float'),
        'text' : pd.Series(dtype='str'),
        'is_retweet': pd.Series(dtype='float'),
        'reply_count' : pd.Series(dtype='float'),
        'quote_count' : pd.Series(dtype='float'),
        'bookmark_count' : pd.Series(dtype='float'),
        'likes': pd.Series(dtype='float'),
        'retweet_count' : pd.Series(dtype='float')
    })

for file in glob.glob(os.path.join('processed_data', '**/*.json'), recursive=True):
    with open(file, 'r', encoding='utf-8') as f:
        df = pd.concat((df, pd.DataFrame(json.load(f), index=[0])), ignore_index=True)

In [6]:
df_agg = df.groupby(by='id').aggregate({
    'username' : 'first',
    'created_at' : 'first',
    'favourites_count': np.mean,
    'followers_count': np.mean,
    'friends_count' : np.mean,
    'created_on' : lambda x: len(x)/(np.max(x) - np.min(x) + 1),
    'is_retweet': np.mean,
    'reply_count' : np.mean,
    'quote_count' : np.mean,
    'bookmark_count' : np.mean,
    'likes': np.mean,
    'retweet_count' : np.mean,
}).rename(columns={
    'created_on' : 'tweet_frequency',
    'is_retweet': 'retweet_rate',
    'reply_count' : 'mean_reply_count',
    'quote_count' : 'mean_quote_count',
    'bookmark_count' : 'mean_bookmark_count',
    'likes': 'mean_likes',
    'retweet_count' : 'mean_retweet_count',
}).reset_index()

C:\Users\dangn\AppData\Local\Temp\ipykernel_3768\1035889834.py:1: FutureWarning: The provided callable <function mean at 0x000001FA70328F70> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_agg = df.groupby(by='id').aggregate({


In [7]:
with open('model', 'rb') as f:
    model = pickle.load(f)

In [8]:
pred = model.predict(df_agg.iloc[:, 3:])
df_out = pd.concat((df_agg, pd.DataFrame({'quality' : pred})), axis=1)

In [9]:
display(df_out)

,id,username,created_at,favourites_count,followers_count,friends_count,tweet_frequency,retweet_rate,mean_reply_count,mean_quote_count,mean_bookmark_count,mean_likes,mean_retweet_count,quality
0,1233895865011593216,NFTDegenNSM,2020-02-29T23:24:51.000Z,8076.000000,107554.0,3437.0,1.431576,0.203390,0.0,0.0,0.152542,101.186441,0.0,0
1,1444655950116401158,nftjuno,2021-10-03T13:30:11.000Z,6113.000000,55208.0,404.0,0.766494,0.066667,0.0,0.0,0.133333,76.900000,0.0,0
2,1674781645810049024,Akraieth,2023-06-30T14:07:45.000Z,21910.333333,2139.0,5001.0,4.849138,0.116667,0.0,0.0,0.000000,27.600000,0.0,0
3,1689966317254631424,benny_ben5,2023-08-11T11:45:56.000Z,1433.000000,77.0,83.0,27.931034,0.966667,0.0,0.0,0.083333,0.133333,0.0,1
4,1702231087110627328,novainscription,2023-09-14T08:02:11.000Z,50.000000,2185.0,56.0,6.151899,0.583333,0.0,0.0,0.027778,2.583333,0.0,2


In [48]:
client = Elasticsearch('https://192.168.0.110:9200', basic_auth=('elastic', '=3BF3=bAwQ7363H9dUdY'), verify_certs=False)

c:\Users\dangn\Documents\trash\lib\site-packages\elasticsearch\_sync\client\__init__.py:397: SecurityWarning: Connecting to 'https://192.168.0.110:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [50]:
client.info()

c:\Users\dangn\Documents\trash\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.110'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'name': 'faa6812381b8', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'kpa1miRwSZuWQ7MO38xtoQ', 'version': {'number': '8.11.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '64cf052f3b56b1fd4449f5454cb88aca7e739d9a', 'build_date': '2023-12-08T11:33:53.634979452Z', 'build_snapshot': False, 'lucene_version': '9.8.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [51]:
for dict in df_out.to_dict(orient='records'):
    client.index(
        index='g20',
        id=dict['id'],
        document={key: dict[key] for key in dict if key != 'id'}
    )

c:\Users\dangn\Documents\trash\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.110'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\dangn\Documents\trash\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.110'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\dangn\Documents\trash\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.110'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\dangn\Documents\trash\lib\site-packag

1
1
1
1
1
